

## Load our dataset :

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.1.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 18.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [73]:
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv,GATConv
from torch_geometric.utils import train_test_split_edges
import torch.nn.functional as F
import numpy as np
from torch_geometric.nn.models import InnerProductDecoder, VGAE
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops
import torch.nn as nn
from torch.optim import Adam
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges


1. adjacency martix

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [74]:
weighted_adjacency_matrix = np.load('/content/drive/My Drive/Data/ProcessedData/World/adjacency matrix/5400/reduced_graph_a=1.5_b=0.5_sp=0.044.npy')

In [75]:
# weighted_adjacency_matrix[weighted_adjacency_matrix < 0.7] = 0

In [76]:
edge_index = np.array(weighted_adjacency_matrix.nonzero())
np.shape(edge_index)

(2, 685176)

2. feature matrix

In [77]:
feature_matrix = np.load('/content/drive/My Drive/Data/ProcessedData/World/features/final feature mat/reduced_feature_mat4.npy')
feature_matrix = np.nan_to_num(feature_matrix)

In [78]:
from torch_geometric.data import Data

In [79]:
# Convert the adjacency matrix to a COO format
edge_index = np.array(weighted_adjacency_matrix.nonzero())#.t()
np.shape(edge_index)

(2, 685176)

In [80]:

# Convert the feature matrix to PyTorch tensor
x = torch.tensor(feature_matrix, dtype=torch.float)

# Create the PyG Data object
data = Data(x=x, edge_index=edge_index)#,edge_weight=np.random.randn(100))

# Print the created Data object
print(data)

Data(x=[3294, 484], edge_index=[2, 685176])


In [81]:
temp = data.edge_index
temp = np.transpose(temp)

In [82]:
edges = []
for i,j in temp:
    if i != j:
        edges.append((i,j))

In [83]:
edges = np.transpose(edges)
edges.shape

(2, 685176)

In [84]:
weights = []
for i,j in np.transpose(data.edge_index):
    # print(i,j)
    if i!=j:
        weights.append(weighted_adjacency_matrix[i,j])


In [85]:
weights = np.array(weights)
weights.shape, weights[0]

((685176,), 0.04350505946947198)

In [86]:
# Convert the feature matrix to PyTorch tensor
x = torch.tensor(feature_matrix, dtype=torch.float)

# Create the PyG Data object
data = Data(x=x, edge_index=torch.tensor(edges),edge_attr=torch.tensor(weights).unsqueeze(1))  #torch.tensor(weights)

# Print the created Data object
print(data)

Data(x=[3294, 484], edge_index=[2, 685176], edge_attr=[685176, 1])


----
----
----
# VGAE

In [87]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.gcn_shared = GATConv(in_channels, hidden_channels)
        # self.gcn_shared2 = GCNConv(hidden_channels, hidden_channels)
        # self.gcn_shared3 = GCNConv(hidden_channels, hidden_channels)
        # self.gcn_shared4 = GCNConv(hidden_channels, hidden_channels)
        # self.gcn_shared5 = GCNConv(hidden_channels, hidden_channels)

        self.gcn_mu = GATConv(hidden_channels, out_channels)
        self.gcn_logvar = GATConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.gcn_shared(x, edge_index))
        # x = F.relu(self.gcn_shared2(x, edge_index))
        # x = F.relu(self.gcn_shared3(x, edge_index))
        # x = F.relu(self.gcn_shared4(x, edge_index))
        # x = F.relu(self.gcn_shared5(x, edge_index))

        mu = self.gcn_mu(x, edge_index)
        logvar = self.gcn_logvar(x, edge_index)
        return mu, logvar


class DeepVGAE(VGAE):
    def __init__(self, num_features, out_channels):
        super(DeepVGAE, self).__init__(encoder=GCNEncoder(num_features,128, out_channels),
                                       decoder=InnerProductDecoder())

    def forward(self, x, edge_index):
        z = self.encode(x, edge_index)
        adj_pred = self.decoder.forward_all(z)
        return adj_pred

    def loss(self, x, pos_edge_index, all_edge_index):
        z = self.encode(x, pos_edge_index)

        pos_loss = -torch.log(
            self.decoder(z, pos_edge_index, sigmoid=True) + 1e-15).mean()

        # Do not include self-loops in negative samples
        all_edge_index_tmp, _ = remove_self_loops(all_edge_index)
        all_edge_index_tmp, _ = add_self_loops(all_edge_index_tmp)

        neg_edge_index = negative_sampling(all_edge_index_tmp, z.size(0), pos_edge_index.size(1))
        neg_loss = -torch.log(1 - self.decoder(z, neg_edge_index, sigmoid=True) + 1e-15).mean()

        kl_loss = 1 / x.size(0) * self.kl_loss()

        return pos_loss + neg_loss + kl_loss

    def single_test(self, x, train_pos_edge_index, test_pos_edge_index, test_neg_edge_index):
        with torch.no_grad():
            z = self.encode(x, train_pos_edge_index)
        roc_auc_score, average_precision_score = self.test(z, test_pos_edge_index, test_neg_edge_index)
        return roc_auc_score, average_precision_score

In [88]:
# torch.manual_seed(12345)
from IPython.display import Javascript  # Restrict height of output cell.
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# parameters
out_channels = 64
num_features = data.num_features
epochs = 200

model = DeepVGAE(num_features, out_channels).to(device)
optimizer = Adam(model.parameters(), lr=0.01)

data = data.to(device)
all_edge_index = data.edge_index
data = train_test_split_edges(data, 0.05, 0.1)

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    loss = model.loss(data.x, data.train_pos_edge_index, all_edge_index)
    loss.backward()
    optimizer.step()
    if epoch % 2 == 0:
        model.eval()
        roc_auc, ap = model.single_test(data.x,
                                        data.train_pos_edge_index,
                                        data.test_pos_edge_index,
                                        data.test_neg_edge_index)
        print("Epoch {} - Loss: {:.3f} ROC_AUC: {:.3f} Precision: {:.3f}".format(epoch, loss.cpu().item(), roc_auc, ap))

<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


Epoch 0 - Loss: 239602.766 ROC_AUC: 0.500 Precision: 0.500
Epoch 2 - Loss: 134.049 ROC_AUC: 0.500 Precision: 0.500
Epoch 4 - Loss: 101.699 ROC_AUC: 0.500 Precision: 0.500
Epoch 6 - Loss: 80.866 ROC_AUC: 0.500 Precision: 0.500
Epoch 8 - Loss: 81.557 ROC_AUC: 0.500 Precision: 0.500
Epoch 10 - Loss: 81.865 ROC_AUC: 0.500 Precision: 0.500
Epoch 12 - Loss: 59.100 ROC_AUC: 0.500 Precision: 0.500
Epoch 14 - Loss: 58.382 ROC_AUC: 0.505 Precision: 0.503
Epoch 16 - Loss: 57.102 ROC_AUC: 0.533 Precision: 0.517
Epoch 18 - Loss: 53.724 ROC_AUC: 0.558 Precision: 0.531
Epoch 20 - Loss: 54.758 ROC_AUC: 0.564 Precision: 0.534
Epoch 22 - Loss: 51.808 ROC_AUC: 0.607 Precision: 0.561
Epoch 24 - Loss: 49.354 ROC_AUC: 0.661 Precision: 0.596
Epoch 26 - Loss: 45.217 ROC_AUC: 0.691 Precision: 0.618
Epoch 28 - Loss: 44.958 ROC_AUC: 0.694 Precision: 0.620
Epoch 30 - Loss: 43.591 ROC_AUC: 0.723 Precision: 0.644
Epoch 32 - Loss: 42.192 ROC_AUC: 0.730 Precision: 0.650
Epoch 34 - Loss: 42.949 ROC_AUC: 0.723 Precisio

In [89]:
Z = model.encode(data.x, data.train_pos_edge_index)
Z.shape

torch.Size([3294, 64])

In [90]:
torch.save(Z, '/content/drive/My Drive/Data/Node Embeddings/node_embedding23.pt')